# AUEB M.Sc. in Data Science

## Course: Large Scale Data Management

### Homework

### Part 2

#### Submission of <mark>Spiros Politis (p3351814)</mark>

---

Import Python packages:

In [ ]:
import string
import itertools
from operator import add
from pyspark import SparkContext

Define functions:

In [ ]:
'''
    Strips a line of text from whitespace, 
    removes punctuation and convert to lowercase.
'''
def cleanup_text(text):
    import string
    
    # Strip the line of text from leading and trailing white space
    text = text.strip()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Convert all words to lowercase
    text = text.lower()
    
    return text


'''
    Generator function that yields a mapper key.
'''
def get_key(line):
    # Split line into words
    words = line.split()

    # Create combinations of words found in the line
    word_combinations = itertools.combinations(words, 2)
    
    for word_combination in word_combinations:
        # Produce a key with format
        # <tupple 1st word>-<tupple 2nd word>
        # Tab (\t) demarkates key, value
        yield word_combination[0] + "," + word_combination[1]

Get or create Spark context:

In [ ]:
spark_context = SparkContext.getOrCreate()

Read text file from HDFS and get a RDD:

In [ ]:
text_rdd = spark_context.textFile("hdfs://namenode/data/shakespeare.txt")

Create Spark job:

In [ ]:
# Cleanup text
# Get key
# Map each key to value 1 
# Reduce by key with add operation
word_count = text_rdd \
    .map(lambda line: cleanup_text(line)) \
    .flatMap(lambda line: get_key(line)) \
    .map(lambda line: (line, 1)) \
    .reduceByKey(add)

Save job output to HDFS:

In [ ]:
word_count.coalesce(1).saveAsTextFile("hdfs://namenode/data/part-2/spark")

Optionally, print output:

In [ ]:
word_count_collect = word_count.collect()

for (word, count) in word_count_collect:
    print("{}\t{}".format(word, count))
    
spark.stop()

---